# Transaction Fraud Analysis

Remember, the dataset can be downloaded at: https://www.kaggle.com/datasets/vardhansiramdasu/fraudulent-transactions-prediction

## Exploratory Analysis:

First, read the dataset into spark and check descriptive statistics

Notes from meeting:
- Use Precision/Recall for metrics, not just high accuracy
- Find feature importances -> factor into prediction, focus on interpretability
- Use bucketing, not z-score to better deal with outliers in the dataset
- Train/test split based on 'step' feature, train on earlier data, test with newer data
- 'step' corresponds to an hour of a day - consider the time of day that the fraud occurs
- Look into first character for nameOrig, determine if transaction originated from customer -> merchant, other way around

In [1]:
# imports, init SparkSession
from pyspark.sql import SparkSession
import pyspark.sql.functions as fn
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

spark = SparkSession.builder.master('local[2]').config("spark.executor.memory", "1g").config("spark.driver.memory", "1g").appName('fraud').getOrCreate()

# read file intto pyspark dataframe
df = spark.read.options(inferSchema = True).csv('Fraud.csv', header=True)
df.printSchema()

root
 |-- step: integer (nullable = true)
 |-- type: string (nullable = true)
 |-- amount: double (nullable = true)
 |-- nameOrig: string (nullable = true)
 |-- oldbalanceOrg: double (nullable = true)
 |-- newbalanceOrig: double (nullable = true)
 |-- nameDest: string (nullable = true)
 |-- oldbalanceDest: double (nullable = true)
 |-- newbalanceDest: double (nullable = true)
 |-- isFraud: integer (nullable = true)
 |-- isFlaggedFraud: integer (nullable = true)



In [3]:
# Descriptive Statistics
pandas_df = pd.read_csv('Fraud.csv')
pandas_df.describe()

,step,amount,oldbalanceOrg,newbalanceOrig,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
count,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06,6.362620e+06
mean,2.433972e+02,1.798619e+05,8.338831e+05,8.551137e+05,1.100702e+06,1.224996e+06,1.290820e-03,2.514687e-06
std,1.423320e+02,6.038582e+05,2.888243e+06,2.924049e+06,3.399180e+06,3.674129e+06,3.590480e-02,1.585775e-03
min,1.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
25%,1.560000e+02,1.338957e+04,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00
50%,2.390000e+02,7.487194e+04,1.420800e+04,0.000000e+00,1.327057e+05,2.146614e+05,0.000000e+00,0.000000e+00
75%,3.350000e+02,2.087215e+05,1.073152e+05,1.442584e+05,9.430367e+05,1.111909e+06,0.000000e+00,0.000000e+00
max,7.430000e+02,9.244552e+07,5.958504e+07,4.958504e+07,3.560159e+08,3.561793e+08,1.000000e+00,1.000000e+00


In [4]:
pandas_df.head()

,step,type,amount,nameOrig,oldbalanceOrg,newbalanceOrig,nameDest,oldbalanceDest,newbalanceDest,isFraud,isFlaggedFraud
0,1,PAYMENT,9839.64,C1231006815,170136.0,160296.36,M1979787155,0.0,0.0,0,0
1,1,PAYMENT,1864.28,C1666544295,21249.0,19384.72,M2044282225,0.0,0.0,0,0
2,1,TRANSFER,181.00,C1305486145,181.0,0.00,C553264065,0.0,0.0,1,0
3,1,CASH_OUT,181.00,C840083671,181.0,0.00,C38997010,21182.0,0.0,1,0
4,1,PAYMENT,11668.14,C2048537720,41554.0,29885.86,M1230701703,0.0,0.0,0,0


### Steps with the most fraudulent charges:

In [5]:
df.groupBy('step').agg({'isFraud':'sum'}).orderBy('sum(isFraud)', ascending=False).show()

+----+------------+
|step|sum(isFraud)|
+----+------------+
| 212|          40|
| 523|          30|
| 387|          28|
| 730|          28|
| 249|          28|
| 501|          28|
| 425|          28|
| 398|          26|
| 160|          26|
| 694|          24|
| 625|          24|
|  66|          24|
| 406|          24|
|  22|          23|
| 296|          22|
| 149|          22|
| 250|          22|
| 279|          22|
|  34|          22|
| 262|          22|
+----+------------+
only showing top 20 rows



In [6]:
df.show(5)

+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|step|    type|  amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|
+----+--------+--------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+
|   1| PAYMENT| 9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|      0|             0|
|   1| PAYMENT| 1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|      0|             0|
|   1|TRANSFER|   181.0|C1305486145|        181.0|           0.0| C553264065|           0.0|           0.0|      1|             0|
|   1|CASH_OUT|   181.0| C840083671|        181.0|           0.0|  C38997010|       21182.0|           0.0|      1|             0|
|   1| PAYMENT|11668.14|C2048537720|      41554.0|      29885.86|M1230701703|      

## Preprocessing -> dummy encodings

In [7]:
# flag transactions from customers to merchants, from customers to customers
# Note: there are no transactions in the dataset originating from merchants!
df1 = df.withColumn('orig_firstletter', df.nameOrig.substr(0,1))
df1 = df1.withColumn('dest_firstletter', df.nameDest.substr(0,1))
df1 = df1.withColumn('Cust_to_Merch', fn.when((df1.orig_firstletter == 'C') & (df1.dest_firstletter == 'M'), 1).otherwise(0))
df1 = df1.withColumn('Cust_to_Cust', fn.when((df1.orig_firstletter == 'C') & (df1.dest_firstletter == 'C'), 1).otherwise(0))
# drop intermediate columns
df1 = df1.drop('orig_firstletter').drop('dest_firstletter')

# account for hour of day from the 'step' feature
df1 = df1.withColumn('hourOfDay', df1.step % 24)


# bucketing: type feature
# type = PAYMENT:  0
# type = CASH_OUT: 1
# type = CASH_IN:  2
# type = TRANSFER: 3
# type = DEBIT:    4
df1 = df1.withColumn('trans_type', fn.when(df1.type == 'PAYMENT', 0)
                    .when(df1.type == 'CASH_OUT', 1)
                    .when(df1.type == 'CASH_IN', 2)
                    .when(df1.type == 'TRANSFER', 3)
                    .when(df1.type == 'DEBIT', 4)
                    .otherwise(5)) # 5 denotes error, should be no 5's
# drop original field
df1 = df1.drop('type')

# TODO: bucketing for oldbalanceOrig, newbalanceOrig, oldbalanceDest, newbalanceDest features



df1.show(50)

+----+---------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+-------------+------------+---------+----------+
|step|   amount|   nameOrig|oldbalanceOrg|newbalanceOrig|   nameDest|oldbalanceDest|newbalanceDest|isFraud|isFlaggedFraud|Cust_to_Merch|Cust_to_Cust|hourOfDay|trans_type|
+----+---------+-----------+-------------+--------------+-----------+--------------+--------------+-------+--------------+-------------+------------+---------+----------+
|   1|  9839.64|C1231006815|     170136.0|     160296.36|M1979787155|           0.0|           0.0|      0|             0|            1|           0|        1|         0|
|   1|  1864.28|C1666544295|      21249.0|      19384.72|M2044282225|           0.0|           0.0|      0|             0|            1|           0|        1|         0|
|   1|    181.0|C1305486145|        181.0|           0.0| C553264065|           0.0|           0.0|      1|             0|            0|         

In [8]:
df1.groupBy('Cust_to_Merch').count().show()

+-------------+-------+
|Cust_to_Merch|  count|
+-------------+-------+
|            1|2151495|
|            0|4211125|
+-------------+-------+



In [9]:
df1.groupBy('Cust_to_Cust').count().show()

+------------+-------+
|Cust_to_Cust|  count|
+------------+-------+
|           1|4211125|
|           0|2151495|
+------------+-------+



In [10]:
df.groupBy('type').count().show()

+--------+-------+
|    type|  count|
+--------+-------+
|TRANSFER| 532909|
| CASH_IN|1399284|
|CASH_OUT|2237500|
| PAYMENT|2151495|
|   DEBIT|  41432|
+--------+-------+



In [11]:
df1.groupBy('trans_type').count().show() # good, we didn't get any '5' fields here, values are consistent

+----------+-------+
|trans_type|  count|
+----------+-------+
|         1|2237500|
|         3| 532909|
|         4|  41432|
|         2|1399284|
|         0|2151495|
+----------+-------+



In [13]:
df1.columns

['step',
 'amount',
 'nameOrig',
 'oldbalanceOrg',
 'newbalanceOrig',
 'nameDest',
 'oldbalanceDest',
 'newbalanceDest',
 'isFraud',
 'isFlaggedFraud',
 'Cust_to_Merch',
 'Cust_to_Cust',
 'hourOfDay',
 'trans_type']